In [48]:
import numpy as np
import re


palabras_a_indice = {}
indices_a_palabras = {}
diccionario_onehot = {}
diccionario_onehot_a_palabra = {}
diccionario_conteo = {}

with open("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\corpus.txt", "r", encoding="utf-8") as f:
    words = f.read().splitlines()

def unir_palabras_en_contexto(corpus, palabra_objetivo1, palabra_objetivo2, contexto=1):
    corpus_modificado = []
    i = 0

    while i < len(corpus):
        palabra = corpus[i]

        # Si encontramos una de las palabras objetivo
        if palabra == palabra_objetivo1 or palabra == palabra_objetivo2:
            # Miramos dentro del rango del contexto (siguiente palabra incluida)
            for j in range(1, contexto + 1):
                if i + j < len(corpus):
                    siguiente = corpus[i + j]

                    if {palabra, siguiente} == {palabra_objetivo1, palabra_objetivo2}:
                        # Respetar orden según aparición
                        token = f"{palabra} {siguiente}"
                        corpus_modificado.append(token)
                        i += j + 1  
                        break
            else:
                
                corpus_modificado.append(palabra)
                i += 1
        else:
            corpus_modificado.append(palabra)
            i += 1

    return corpus_modificado

palabras_a_unir = [
                    (',','y'),
                    ('.','no'),
                    ('. ','no'),
                    ('que', 'lo'),
                    ('de','la'),
                    ('a','la'),
                    ('en','el'),
                    ('pero',','),
                    ('lo','que'),
                    ('la','de'),
                    ('con',','),
                    ('se','que'),
                    ('como', 'un'),
                    ('por','la'),
                    ('o',','),
                    ('no','?'),
                    ('para','que'),
                    (';','el'),
                    ('la',':')
]

corpus_modificado = words.copy()

# Función para unir las palabras en contexto
for palabra1, palabra2 in palabras_a_unir:
    # Llamamos a la función para unir las palabras en el corpus actual
    corpus_modificado = unir_palabras_en_contexto(corpus_modificado, palabra1, palabra2, 1)

    
for token in corpus_modificado:
    if token not in palabras_a_indice:
        index = len(palabras_a_indice)
        palabras_a_indice[token] = index
        indices_a_palabras[index] = token
        diccionario_conteo[token] = 1 # Initialize count for new token
    else:
        diccionario_conteo[token] += 1 # Increment count for existing token


cardinal_V = len(palabras_a_indice)

for token, idx in list(palabras_a_indice.items()):

    one_hot_vector = np.zeros(cardinal_V)
    one_hot_vector[idx] = 1
    diccionario_onehot[token] = one_hot_vector
    diccionario_onehot_a_palabra[str(one_hot_vector)] = token

In [51]:
len(words)

310347

In [52]:
len(set(words))

27971

In [53]:
len(set(corpus_modificado))

27998

In [50]:
len(corpus_modificado)

300222

In [56]:
def cargar_modelo_completo(nombre_archivo='pesos_cbow_pc2_epoca0.npz'):
    """
    Carga los pesos W1, W2 y los hiperparámetros N, C y eta
    desde un archivo .npz.
    """
    try:
        data = np.load(nombre_archivo)

        W1 = data['W1']
        W2 = data['W2']

        N = data['N'].item()
        C = data['C'].item()
        eta = data['eta'].item()

        print()

        return W1, W2,N, C, eta

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'.")
        return None, None, None, None, None


In [94]:
W1, W2,N, C, eta = cargar_modelo_completo("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\pesos_cbow_neg_epoca900_contexto_5.npz")


In [95]:
C

5

In [59]:
def generar_tuplas_nuevo(corpus, palabras_a_indice, contexto):

    indices = [i for i in range(contexto,(len(corpus)-contexto))]

    indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]

    indices_tuplas = []

    contexto_a_central = {} 

    for i in indices:

        palabra_central = palabras_a_indice[corpus[i]]

        contexto_actual = tuple(palabras_a_indice[corpus[i+j]] for j in indices_contexto)

        if contexto_actual in contexto_a_central:
            if contexto_a_central[contexto_actual] != palabra_central:
                continue 
        else:
            contexto_a_central[contexto_actual] = palabra_central

        indices_tuplas.append((palabra_central, list(contexto_actual)))

    return indices_tuplas


len(generar_tuplas_nuevo(corpus_modificado,palabras_a_indice,2))

297559

In [6]:
def generar_tuplas(corpus, palabras_a_indice, contexto):

    ## Genero una lista de todos los indices de las palabras en el corpus, posibles sin padding
    indices = [i for i in range(contexto,(len(corpus)-contexto))]

    ## Genero una lista de los contextos de las palabras de contexto
    indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]

    indices_tuplas = []

    ## Por cada indice en indices, genero una tupla (indice_central, [indices_contexto])
    for i in indices:

        indices_tuplas.append((palabras_a_indice[corpus[i]], [palabras_a_indice[corpus[i+j]] for j in indices_contexto]))

    return indices_tuplas   

len(generar_tuplas(words,palabras_a_indice,2))

310343

In [90]:
#! pip install cupy-cuda11x


def softmax(x):
    x = np.asarray(x)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predecir_cbow(palabra_a_indices, indices_a_palabras, W1, W2, contexto_indices, topk=5):

    h = np.mean(W1[contexto_indices], axis=0).reshape(-1, 1)

    u = W2.T @ h
    y = sigmoid(u).flatten()
    #y = softmax(u).flatten()
    # --- Opción alternativa: similitud coseno ---
    # Normalizamos vectores de palabras y contexto
    #W2_norm = W2 / np.linalg.norm(W2, axis=0, keepdims=True)
    #h_norm = h / np.linalg.norm(h)
    #similitudes = (W2_norm.T @ h_norm).flatten()
    #candidatos = np.argsort(-similitudes)

    candidatos = np.argsort(-y)
    palabra_top1 = indices_a_palabras[candidatos[0]]
    palabras_topk = [indices_a_palabras[i] for i in candidatos[:topk]]

    return palabra_top1, palabras_topk


def predecir_cbow_completo(palabra_a_indices, indices_a_palabras, corpus, W1, W2, contexto, topk=5):
    
    tuplas = generar_tuplas_nuevo(corpus, palabra_a_indices, contexto)
    aciertos_top1 = 0
    aciertos_topk = 0
    palabras_mal_predichas = []

    totales = len(tuplas)

    for i, (p_objetivo, p_contexto) in enumerate(tuplas):
        palabra_top1, palabras_topk = predecir_cbow(
            palabra_a_indices, indices_a_palabras, W1, W2, p_contexto, topk=topk
        )

        # Top-1
        if indices_a_palabras[p_objetivo] == palabra_top1:
            aciertos_top1 += 1

        # Top-k
        if indices_a_palabras[p_objetivo] in palabras_topk:
            aciertos_topk += 1
        else:
            palabras_mal_predichas.append(indices_a_palabras[p_objetivo])

        # Progreso
        if i % 100 == 0 and i > 0:
            progreso = (i / totales) * 100
            print(f"Progreso: [{progreso:.1f}%] - Top1: {aciertos_top1/totales:.4f}%, Top{topk}: {aciertos_topk/totales:.4f}%", end='\r')

    accuracy_top1 = aciertos_top1 / totales
    accuracy_topk = aciertos_topk / totales

    print(f"\nResultados completos: Top1={accuracy_top1*100:.2f}%, Top{topk}={accuracy_topk*100:.2f}%")

    return accuracy_top1, accuracy_topk, palabras_mal_predichas

In [ ]:
accuracy_top1, accuracy_topk, palabras_mal_predichas = predecir_cbow_completo(palabras_a_indice,indices_a_palabras,corpus_modificado,W1,W2,5,10)


In [96]:
print(f"Accuracy flexible: {accuracy_topk}")
print(f"Error flexible: {1 - accuracy_topk}")
print()
print(f"Accuracy: {accuracy_top1}")
print(f"Error: {1 - accuracy_top1}")

Accuracy flexible: 0.11581910552177896
Error flexible: 0.884180894478221

Accuracy: 0.024602315316065628
Error: 0.9753976846839344


In [ ]:
lista = palabras_mal_predichas
len(set(lista))

27967

In [ ]:
cuenta = {}

for i in lista:
    if i not in cuenta:
        cuenta[i] = 1
    else:
        cuenta[i] += 1

In [ ]:
import pandas as pd

cuenta = pd.Series(cuenta)
cuenta = cuenta.sort_values(ascending=False)


cuenta.head(50)


,           20351
de          13247
la           9968
.            9726
que          9048
y            8760
a            6627
el           6528
en           6114
se           3964
.            3784
un           3717
no           3565
los          3237
una          2719
lo           2517
con          2408
por          2361
del          2306
las          2304
es           1745
como         1633
me           1562
para         1536
pero         1499
le           1480
al           1462
su           1258
más          1203
o            1171
dijo         1073
era           924
había         905
si            904
oliveira      877
yo            793
todo          771
?             766
te            747
qué           705
ya            699
;             657
sin           637
eso           622
cuando        593
:             582
sus           571
poco          517
hasta         516
porque        514
dtype: int64

In [ ]:



def palabras_frecuentes_en_contexto(corpus, palabra_objetivo, contexto=1):

    frecuencias = {}  
    
    for i in range(len(corpus)):
        
        if corpus[i] == palabra_objetivo:
            
            for j in range(i - contexto, i + contexto + 1):
                
                if j != i and 0 <= j < len(corpus):  
                    palabra_contexto = corpus[j]
                    
                    if palabra_contexto in frecuencias:
                        frecuencias[palabra_contexto] += 1
                    else:
                        frecuencias[palabra_contexto] = 1
    
    
    return sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

with open('frecuencia_de_salida', 'a', encoding='utf-8') as f:
    for palabra in cuenta.index:
    
        palabra_objetivo = palabra
        resultados = palabras_frecuentes_en_contexto(words, palabra_objetivo, 1)
        f.write(f"Palabras más frecuentes cerca de '{palabra_objetivo}',que fue mal predicha :{cuenta[palabra]}\n")
        print("Palabras más frecuentes cerca de '{}':".format(palabra_objetivo))
        for palabra, frecuencia in resultados[:5]:
            f.write(f"{palabra}: {frecuencia}\n")
            print(f"{palabra}: {frecuencia}")
        f.write("\n" + "="*50 + "\n")

Palabras más frecuentes cerca de '.':
.: 1910
no: 421
pero: 324
la: 324
y: 318
Palabras más frecuentes cerca de ',':
y: 1992
pero: 755
la: 696
que: 609
el: 576
Palabras más frecuentes cerca de 'la':
de: 1854
en: 1101
a: 977
,: 696
maga: 463
Palabras más frecuentes cerca de 'de':
la: 1854
los: 659
,: 484
que: 475
las: 466
Palabras más frecuentes cerca de 'que':
lo: 788
,: 609
se: 571
no: 503
de: 475
Palabras más frecuentes cerca de 'el':
en: 1219
,: 576
que: 475
y: 411
.: 314
Palabras más frecuentes cerca de 'a':
la: 977
,: 417
.: 253
y: 249
los: 245
Palabras más frecuentes cerca de 'y':
,: 1992
el: 411
la: 383
se: 323
.: 318
Palabras más frecuentes cerca de 'en':
el: 1219
la: 1101
,: 435
que: 394
un: 336
Palabras más frecuentes cerca de 'se':
que: 571
,: 374
no: 364
y: 323
.: 146
Palabras más frecuentes cerca de '. ':
no: 227
.: 204
oliveira: 116
la: 113
por: 112
Palabras más frecuentes cerca de 'un':
de: 393
,: 346
en: 336
poco: 298
con: 232
Palabras más frecuentes cerca de 'los':
de:

In [131]:
len(corpus_modificado)

310347

In [132]:
len(words)

310347

In [133]:
len(set(corpus_modificado))

27988

In [134]:
len(set(words))

27971